In [ ]:
from PIL import Image, ImageOps
import requests
import io
import math
from tqdm import tqdm
import numpy as np

url = "https://assets.pokemon.com/assets/cms2/img/pokedex/detail/428.png"

pokemon_count = 1025

data = np.memmap(
    "data.bin",
    dtype=np.uint8,
    mode="w+",
    shape=pokemon_count * 512,
)

for pokemon_index in tqdm(range(1, pokemon_count + 1), desc="Writing Pokemon Data"):
    start_index = (pokemon_index - 1) * 512
    url = f"https://assets.pokemon.com/assets/cms2/img/pokedex/detail/{pokemon_index:03}.png"
    response = requests.get(url, stream=True, timeout=1000)
    image = Image.open(io.BytesIO(response.content))
    bbox = image.getbbox()
    if bbox:
        image = image.crop(bbox)
    image = image.resize((37, 25))
    image = image.convert(mode="P", palette=Image.ADAPTIVE, colors=16)
    palette = np.array(image.getpalette(), dtype="uint8")
    # print(palette.shape)
    # print(data[start_index : start_index + 16 * 3].shape)
    data[start_index : start_index + 16 * 3] = palette
    start = start_index + 16 * 3
    end = start + 37 * 25 // 2

    image_data = np.array(image, dtype="uint8").reshape((-1))
    data[start : end + 1] = image_data[0::2] << 4
    data[start:end] |= image_data[1::2] & 0xF

    # print(palette)
    # print(palette.reshape((-1, 3)))
    # print(palette.view("uint32"))

Writing Pokemon Data: 100%|██████████| 1025/1025 [04:42<00:00,  3.63it/s]


In [2]:
pixels = 37 * 25
colors = 16
section_bytes = colors * 3 + (pixels * 0.5+1.5)

section_bytes


# pixels/2

512.0

In [8]:
import numpy as np
import base64
data = np.memmap(
    "data.bin",
    dtype=np.uint8,
    mode="r",
)

with open("./src/programs/pokemon_data.ts","wb") as file:
    file.write(b"export const POKEMON_DATA=\"")
    file.write(base64.b64encode(data))
    file.write(b"\";")

In [ ]:
# import gzip

gzip_data = gzip.compress(data)

with open("./src/programs/pokemon_data.ts","w") as file:
    file.write("export const POKEMON_DATA=\"")
    for byte in gzip_data:
        file.write(f"\\u00{byte:02X}")
    file.write("\";")

print(len(gzip_data),len(data),len(gzip_data)/len(data))

NameError: name 'gzip' is not defined

In [1]:
512*(428-1)

218624